In [1]:
import pandas as pd
import numpy as np
from layout import Layout
import networkx as nx

In [2]:
colnames = ["HD", "RAh", "RAm", "RAs", "DE-", "DEd", "DEm", "DEs", "mag"]
colspecs = [(25,31), (75,77), (77,79), (79,83), (83,84), (84,86), (86,88), (88,90), (102,107)]
stars = pd.read_fwf("hd_catalogue.txt", colspecs=colspecs, header=None, names=colnames).dropna()

In [3]:
constellations = pd.read_csv("constellations.csv", sep=";")

In [4]:
conversion = pd.read_csv("hip_hd_conversion.csv")[["hip", "hd"]].dropna().set_index("hip")

In [5]:
def link_list(row):
    n = row["Links"]
    line = row["Stars"]
    ids = line.split(" ")
    array = np.array([int(conversion.loc[int(id), "hd"]) for id in ids])
    return array.reshape((n,2))

constellation_edges = constellations.apply(link_list, axis=1)

In [6]:
G = nx.Graph()
G.add_nodes_from(stars["HD"])
for edges in constellation_edges:
    G.add_edges_from(edges)
nx.info(G)

'Graph with 9096 nodes and 676 edges'

In [7]:
def toXYZ(row):
    sign = -1 if row["DE-"] == "-" else 1
    d = sign * np.pi * (row["DEd"] / 180 + row["DEm"] / 10_800 + row["DEs"] / 648_000)
    a = np.pi * (row["RAh"] / 12 + row["RAm"] / 720 + row["RAs"] / 43200)
    x = np.cos(d) * np.cos(a)
    y = np.cos(d) * np.sin(a)
    z = np.sin(d)
    return np.array([x, y, z])

pos = dict(zip(stars["HD"], stars.apply(toXYZ, axis=1)))

In [8]:
layout = Layout(G, pos)

In [9]:
max_alpha = 255
min_alpha = 0
mag_max = stars["mag"].max()
mag_min = stars["mag"].min()

def get_glow(mag):
    k = (max_alpha - min_alpha) / (mag_min - mag_max)
    d = min_alpha - k * mag_max
    return int(np.round(k * mag + d))

layout.node_colors = np.array([[255, 255, 255, get_glow(mag)] for mag in stars["mag"]])

In [10]:
layout.edge_colors = np.repeat([(255, 255, 255, 100)], repeats=G.number_of_edges(), axis=0)

In [11]:
layout.node_table

,x,y,z,r,g,b,a,name
0,0.852155,0.507935,0.854988,255,255,255,34,node_0
1,0.999989,0.511045,0.495589,255,255,255,45,node_1
2,0.997517,0.511581,0.450248,255,255,255,91,node_2
3,0.986374,0.512096,0.615834,255,255,255,66,node_3
4,0.761716,0.507156,0.926063,255,255,255,54,node_4
...,...,...,...,...,...,...,...,...
9091,0.647086,0.502894,0.022029,255,255,255,18,node_9091
9092,0.911295,0.508787,0.784365,255,255,255,50,node_9092
9093,0.659218,0.503256,0.025933,255,255,255,64,node_9093
9094,0.946905,0.509619,0.724268,255,255,255,46,node_9094


In [12]:
layout.edge_table

,i,j,r,g,b,a
0,14,162,255,255,255,100
1,14,8760,255,255,255,100
2,14,38,255,255,255,100
3,20,165,255,255,255,100
4,24,96,255,255,255,100
...,...,...,...,...,...,...
671,8848,8922,255,255,255,100
672,8896,8969,255,255,255,100
673,8901,8954,255,255,255,100
674,8954,9057,255,255,255,100


In [13]:
layout.write(node_file_path="night_sky_nodes.csv", edge_file_path="night_sky_edges.csv")